## Library Imports


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pandas as pd
sns.set_style('darkgrid')
plt.style.use('seaborn-v0_8-pastel')

import warnings
warnings.simplefilter('ignore')

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
data = pd.read_csv("UrbanEdgeApparel.csv")
data.head()